In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Helper functions
def assign_color(color_identity):
    if (len(color_identity) == 0):
        return "colorless"
    elif (len(color_identity) > 1):
        return "multicolor"
    elif (color_identity == ['W']):
        return "white"
    elif (color_identity == ['U']):
        return "blue"
    elif (color_identity == ['B']):
        return "black"
    elif (color_identity == ['R']):
        return "red"
    elif (color_identity == ['G']):
        return "green"

In [12]:
scanned_cards = (
    pd.read_csv("../data/all-folders.csv", skiprows=1)
    .loc[lambda d: ~d["Folder Name"].isin(["Magic Box (current)", "15 euro MP (ontdubbeld)"])]
    .assign(Value=lambda d: d["MARKET"].astype(float))
    .assign(ID=lambda d: d["Set Code"] + "_" + d["Card Number"])
    .assign(TotalQuantity=lambda d: d.groupby("ID")["Quantity"].transform("sum"))
    .assign(BoxedQuantity = lambda d: d.loc[lambda d: d["Folder Name"].isin(["Magic Binder", "Magic Box", "Oude kaarten", "The Lord of the Rings"])].groupby("ID")["Quantity"].transform("sum"))
    .assign(BoxedQuantity = lambda d: d.groupby("ID")["BoxedQuantity"].transform("sum"))
    .assign(OtherQuantity = lambda d: d.loc[lambda d: ~d["Folder Name"].isin(["Magic Binder", "Magic Box", "Oude kaarten", "The Lord of the Rings"])].groupby("ID")["Quantity"].transform("sum"))
    .assign(OtherQuantity = lambda d: d["OtherQuantity"] - d["Quantity"])
    .assign(
        NormalQuantity=lambda d: d.loc[lambda d: d["Printing"] == "Normal"]
        .groupby("ID")["Quantity"]
        .transform("sum")
    )
    .assign(
        FoilQuantity=lambda d: d.loc[
            lambda d: d["Printing"].isin(["Foil", "Etched Foil"])
        ]
        .groupby("ID")["Quantity"]
        .transform("sum")
    )
    .assign(
        image_url=lambda d: "https://cards.scryfall.io/normal/front/c/4/c40f60fe-0f81-43a8-9205-57cf718db81e.jpg"
    )
    .sort_values("Card Name")
    .fillna(0)
)

In [13]:
# Get 'default cards' from https://scryfall.com/docs/api/bulk-data
card_info = (
    pd.read_json("../data/default-cards.json")
    .assign(set_code = lambda d: d["set"].str.upper())
    .assign(join_id = lambda d: d["set_code"] + "_" + d["collector_number"])
)

In [14]:
my_cards = (
    scanned_cards
    .merge(card_info, left_on="ID", right_on="join_id")
    .assign(color=lambda d: d['color_identity'].apply(assign_color))
    .assign(card_type = lambda d: d["type_line"].apply(lambda x: x.split(" — ")[0]))
    .assign(card_subtype = lambda d: d["type_line"].apply(lambda x: x.split("//")[0].split(" — ")[1] if " — " in x else x.split(" — ")[0]))
    .assign(
        cardface_images=lambda df: np.where(
            df["card_faces"].isna(),
            df["card_faces"],
            df["card_faces"].str[0].str["image_uris"]#.str["png"]
        )
    )
    .assign(
        image_uris=lambda df: np.where(
            df["image_uris"].isna(),
            df["cardface_images"],
            df["image_uris"]
        )
    )
    [[
    "Folder Name",
    "Quantity",
    "Card Name",
    "Set Code",
    "Set Name",
    "Card Number",
    "Printing",
    "Value",
    "ID",
    "TotalQuantity",
    "NormalQuantity",
    "FoilQuantity",
    "BoxedQuantity",
    "OtherQuantity",
    "image_url",
    "id",
    "name",
    "lang",
    "released_at",
    "uri",
    "scryfall_uri",
    "layout",
    "highres_image",
    "image_status",
    "card_faces",
    "image_uris",
    "mana_cost",
    "cmc",
    "type_line",
    "card_type",
    "card_subtype",
    "oracle_text",
    "power",
    "toughness",
    "color",
    "colors",
    "color_identity",
    "keywords",
    "foil",
    "nonfoil",
    "set_id",
    "set",
    "set_name",
    "set_type",
    "set_uri",
    "collector_number",
    "rarity",
    "flavor_text",
    "border_color",
    "frame",
    "prices",
    "set_code",
    "join_id"]]
)

In [15]:
# Add image url columns
expanded_urls = pd.json_normalize(my_cards['image_uris'])
my_cards = pd.concat([my_cards, expanded_urls], axis=1).assign(image_url = lambda d: d["normal"])

In [16]:
my_cards.to_csv("../data/my-cards.csv", index=False)

In [17]:
dragonshield_import_df = (
    scanned_cards
    .assign(dubbel_status = lambda d: np.where(d["BoxedQuantity"] > 0, "Alles dubbel", np.where(d["Quantity"] > 1, "1 houden", "Nieuw")))
    .assign(naar_magic_box = lambda d: np.where(d["dubbel_status"].isin(["Nieuw", "1 houden"]), 1, 0))
    .assign(naar_dubbel = lambda d: np.where(d["dubbel_status"] == "Alles dubbel", d["Quantity"], np.where(d["dubbel_status"] == "1 houden", d["Quantity"] - 1, 0)))
    [[
       'Folder Name', 'Quantity', 'Trade Quantity', 'Card Name', 'Set Code',
       'Set Name', 'Card Number', 'Condition', 'Printing', 'Language',
       'Price Bought', 'Date Bought', 'dubbel_status', "naar_magic_box", "naar_dubbel"
    ]]
)

In [18]:
dragonshield_import_df.to_csv("../data/dragonshield-import.csv", index=False)

In [19]:
dubbel = pd.read_csv("../data/dubbel.csv").assign(copy = lambda d: d["Quantity"].astype(str) + " " + d["Card Name"])

In [20]:
dubbel["copy"].to_csv("../data/verkooplijst.csv", index=False)